<a href="https://colab.research.google.com/github/TayTechie07/Email-Spam-Classification-ML/blob/main/Machine_Learning%E2%80%93Based_Email_Spam_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import string

In [3]:
##loading the dataset
df = pd.read_csv("spam.csv", encoding="latin-1")

df.head()


,Unnamed: 0,spamORham,Message
0,0,ham,"Go until jurong point, crazy.. Available only ..."
1,1,ham,Ok lar... Joking wif u oni...
2,2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,ham,U dun say so early hor... U c already then say...
4,4,ham,"Nah I don't think he goes to usf, he lives aro..."


Dropped Unnamed as it was just a index column

In [5]:
df = df.drop(columns=['Unnamed: 0'])


In [6]:
df = df.rename(columns={
    'spamORham': 'label',
    'Message': 'text'
})

df.head()


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df['label'] = df['label'].map({
    'ham': 0,
    'spam': 1
})


In [8]:
print("Dataset shape:", df.shape)
print("\nLabel distribution:")
print(df['label'].value_counts())


Dataset shape: (5572, 2)

Label distribution:
label
0    4825
1     747
Name: count, dtype: int64


# Text Preprocessing
Remove puntuation , numbers and lowercase the text

In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

df['clean_text'] = df['text'].apply(clean_text)


# TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=3000
)

X = tfidf.fit_transform(df['clean_text'])
y = df['label']


# Train-Test-Split

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [13]:
##Training the model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [14]:
y_pred = model.predict(X_test)


# Model Evaluation

In [15]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9730941704035875


In [16]:
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))



Confusion Matrix:

[[965   0]
 [ 30 120]]


# Classified spam or non spam
Testing the model with custom email data


In [18]:
sample_email = ["Congratulations! You won a free prize. Click now"]

sample_clean = [clean_text(sample_email[0])]
sample_vector = tfidf.transform(sample_clean)

prediction = model.predict(sample_vector)

print("The Email is Spam" if prediction[0] == 1 else "The Email is not Spam")


The Email is Spam
